<a href="https://colab.research.google.com/github/Guillermo-Torrealba/poyecto_gf/blob/master/Hito_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno deriva la formula discreta de valoración de Black-Scholes para un periodo.
Usaremos la librería simbólica para la derivación algebraica general

In [2]:
from sympy import *
init_printing()

La estructura financiera es la siguiente:

Dos periodos $t=0,1$ y 2 estados de la naturaleza en $t=1$: bueno (u) y malo (d)

Dos activos:
- libre de riesgo (bono) con valor inicial $B$ y retorno $\tilde r>1$
- riesgoso (acción) con valor inicial $S_0$ y retornos $u>1$ ó $1>d$

Derivado: opción de compra Europea de la acción (se ejerce solo al final)
- $K$ precio de ejercicio
- $S_1$ valor del activo al final
- $\max\{0,S_1-K\}$ retorno para $S_1\in\{uS_0,dS_0\}$

1. ¿Cuál es el precio de los activos no derivados?
- De esta forma el precio del bono es $B$ y el precio de la acción es $S_0$.
2. ¿Cuál es el retorno de los activos no derivados?
- El retorno del bono es $\tilde rB$ tanto en el estado bueno como en el estado malo
- El retorno de la acción es $uS_0$ en el estado bueno y $dS_0$ en el estado malo
3. ¿Cuál es el retorno de la opción?
- RESPONDER

Inicialmente nos concentraremos en los activos no derivados. Definimos $W=\left[-q'\atop R\right]$ donde los precios de los activos son $q=(q_1,q_2)=(B,S_0)$ y los retornos son $R=(R_1,R_2)$.
1. Encuentre la expresión de $R_1$ y $R_2$.
- RESPONDER
2. Encuentre $W$.
- RESPONDER


In [3]:
# Parámetros Bono
r, B = symbols('r B')
# Parámetros Acción
u, d, S0 = symbols('u d S0')

In [4]:
W = Matrix([[-B, -S0], [r*B, u*S0],[r*B, d*S0]])
W

⎡-B   -S₀ ⎤
⎢         ⎥
⎢B⋅r  S₀⋅u⎥
⎢         ⎥
⎣B⋅r  S₀⋅d⎦

El Teorema Fundamental de Valoración de Activos ([Harrison and Kreps (1979)](https://www.sciencedirect.com/science/article/pii/0022053179900437)) establece que la estructura financiera  es libre de arbitraje si y sólo si existe un $\pi\in\mathbb{R}^S_{++}$ tal que

(Condición) $[1,\pi']W=0$.

Para utilizar este resultado, definamos el vector $[1,\pi']$.

In [5]:
pi_u, pi_d = symbols('pi_u pi_d')
pi = Matrix([[1],[pi_u],[pi_d]])
pi

⎡ 1 ⎤
⎢   ⎥
⎢πᵤ ⎥
⎢   ⎥
⎣π_d⎦

Reemplazando en la (Condición) obtenemos el siguiente sistema de ecuaciones:

In [6]:
NA=pi.transpose()*W
NA

[B⋅π_d⋅r + B⋅πᵤ⋅r - B  S₀⋅d⋅π_d + S₀⋅πᵤ⋅u - S₀]

Podemos utilizar un _solver_ algebraico para encontrar expresiones generales de $\pi_1$ y $\pi_2$ en función de los parámetros $u,d$ y $r$:

In [7]:
# Solver tutorial
# https://docs.sympy.org/latest/modules/solvers/solvers.html
piNA=solve([NA[0],NA[1]],[pi_u,pi_d],dict=True)
piNA

⎡⎧       -r + u          -d + r  ⎫⎤
⎢⎨π_d: ──────────, πᵤ: ──────────⎬⎥
⎣⎩     -d⋅r + r⋅u      -d⋅r + r⋅u⎭⎦

## Inclusión del derivado

Ahora extendamos la matriz $W$ para incorporar la opción.

1. Encuentre la expresión para $W$ agregando los retornos y el precio de la opción denotado por $q_3$.

In [8]:
# Parámetros Opción
q3 = 3.55
K = 185
# Parámetros Bono
r, B = 5.18, 91.77
# Parámetros Acción
u, d, S0 = 1.003, 0.996, 184.16
W = Matrix([[-B, -S0, -q3], [r*B, u*S0, max(0,u*S0-K)],[r*B, d*S0, max(0,d*S0-K)]])
W

⎡ -91.77    -184.16   -3.55⎤
⎢                          ⎥
⎢475.3686  184.71248    0  ⎥
⎢                          ⎥
⎣475.3686  183.42336    0  ⎦

Utilice la (Condición) del Teorema Fundamental reemplazando los valores encontrados anteriormente para $\pi$ para definir una expresión para $q_3$.

In [9]:
u,r,d = symbols('u r d')
Prices=Matrix([[1],[piNA[0][pi_u]],[piNA[0][pi_d]]]).transpose()*W
Prices

⎡475.3686⋅(-d + r)   475.3686⋅(-r + u)          184.71248⋅(-d + r)   183.42336⋅(-r + u)            ↪
⎢───────────────── + ───────────────── - 91.77  ────────────────── + ────────────────── - 184.16   ↪
⎣   -d⋅r + r⋅u          -d⋅r + r⋅u                  -d⋅r + r⋅u           -d⋅r + r⋅u                ↪

↪      ⎤
↪ -3.55⎥
↪      ⎦

In [10]:
Prices.subs([(d,0.5),(u,1.5),(r,0.01)])

[47445.09  18095.00912  -3.55]

## Aplicación (Grupos)

Encuentre valores de $r,B,u,d,S_0$ para valorar una opción.

# Anexo
Para más manipulaciones simbólicas ver [este cuaderno.](https://colab.research.google.com/github/jrjohansson/scientific-python-lectures/blob/master/Lecture-5-Sympy.ipynb#scrollTo=GzKR97oUjW8T)